#Create a model for predict car price

[link to the dataset](https://www.kaggle.com/datasets/doaaalsenani/usa-cers-dataset)

In [ ]:
#import dataframe
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Semendyeav/datasets/main/PDA110_USA_cars_datasets.csv')
df.head(3)

,Unnamed: 0,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,jtezu11f88k007763,159348797,new jersey,usa,10 days left
1,1,2899,ford,se,2011,clean vehicle,190552.0,silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,6 days left
2,2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,3c4pdcgg5jt346413,167655728,georgia,usa,2 days left


In [ ]:
#look at object columns
df.describe(include = 'object')

,brand,model,title_status,color,vin,state,country,condition
count,2499,2499,2499,2499,2499,2499,2499,2499
unique,28,127,2,49,2495,44,2,47
top,ford,door,clean vehicle,white,1gnevhkw8jj148388,pennsylvania,usa,2 days left
freq,1235,651,2336,707,2,299,2492,832


In [ ]:
#delete the columns that will not affect the target
df.drop(['condition', 'vin', 'country', 'title_status', 'lot', 'Unnamed: 0'], inplace = True, axis = 1)
df.head(2)

,price,brand,model,year,mileage,color,state
0,6300,toyota,cruiser,2008,274117.0,black,new jersey
1,2899,ford,se,2011,190552.0,silver,tennessee


In [ ]:
#Delete outliners
Q1 = df['price'].quantile(q=.25)
Q3 = df['price'].quantile(q=.75)
IQR = Q3-Q1

mask = (df['price'] > (Q1-1.5*IQR)) & (df['price'] < (Q3+1.5*IQR))
df = df[mask]

In [ ]:
!pip install category_encoders

In [ ]:
df.head(2)

,price,brand,model,year,mileage,color,state
0,6300,toyota,cruiser,2008,274117.0,black,new jersey
1,2899,ford,se,2011,190552.0,silver,tennessee


In [ ]:
#convert category to numbers use catboost encoder
import category_encoders as ce
# Define train and target
target = df['price']
train = df.drop(['price','year','mileage'], axis = 1)

# Define catboost encoder
cbe_encoder = ce.cat_boost.CatBoostEncoder()
cbe_encoder.fit(train, target)
train_cbe = cbe_encoder.transform(train)
df = df[['price','year','mileage']].join(train_cbe)
df.head(2)

,price,year,mileage,brand,model,color,state
0,6300,2008,274117.0,17796.126078,17796.126078,18771.973992,16227.740069
1,2899,2011,190552.0,20206.198586,17796.126078,16931.869111,16316.486151


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import uniform, randint
# split data
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis = 1), df['price'], test_size=0.2)

# create a model
model = RandomForestClassifier()

#tune hyperparameters of model
hyperparameters = {"n_estimators": randint(0, 1000),
                   "max_depth": randint(1, 4),
                   }
random_search = RandomizedSearchCV(
    model, hyperparameters, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring=['neg_mean_absolute_error'], refit='neg_mean_absolute_error'
)
random_search.fit(X_train, y_train)

print("Best hyperparameters: ", random_search.best_params_)
print("Best score: ", -random_search.best_score_)
# # specify the training parameters
# model = RandomForestClassifier(n_estimators=200, max_depth=3)
# #train the model
# model.fit(X_train,y_train)
# # make the prediction using the resulting model
# preds = model.predict(X_test)
# print(np.mean(np.abs(preds-y_test)))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best hyperparameters:  {'max_depth': 3, 'n_estimators': 435}
Best score:  5481.986847274405


In [ ]:
import numpy as np
model = RandomForestClassifier(n_estimators = 435, max_depth =3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("MAE: ", np.round(np.mean(np.abs(y_pred-y_test)),0))

MAE:  5887.0


In [ ]:
#Save a model
import pickle
pickle_out = open("model_car_dataset.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [ ]:
#Save an encoder
import pickle
pickle_out = open("encoder_car_dataset.pkl","wb")
pickle.dump(cbe_encoder, pickle_out)
pickle_out.close()

**Conclusion:**
1. The performance of the model is not the best. You can use a different model to improve performance.
2. You can define in more detail the features that affect the target and delete not informative
3. You can try another encoder
4. You can draw graphs to look at the data